In [1]:
# Initialise
%run ~/relmapping/annot/notebooks/__init__.ipynb

/mnt/home3/jj374/anaconda36/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


os.getcwd(): /mnt/b2/scratch/ahringer/jj374/lab/relmapping


In [2]:
# Load new annotation & test known corner cases
fp_regl = 'annot/S2_regulatory_annotation/S2_regulatory_annotation.tsv'
df_regl = pd.read_csv(fp_regl, sep='\t')
print('%d regions loaded' % (len(df_regl),))

def check_p(igvstr, annot_rev=None, annot_fwd=None, locus_rev=None, locus_fwd=None):
    (chrom, start, end) = yp.parse_igvstr(igvstr)
    df_ = df_regl.query("chrom == '%s' & start == %d & end == %d" % (chrom, start, end))
    assert len(df_) == 1

    annot_rev_ = df_['annot_rev'].tolist()[0]
    annot_fwd_ = df_['annot_fwd'].tolist()[0]
    locus_rev_ = df_['promoter_locus_id_rev'].tolist()[0]
    locus_fwd_ = df_['promoter_locus_id_fwd'].tolist()[0]

    try:
        assert annot_rev is None or (annot_rev == annot_rev_)
        assert locus_rev is None or (locus_rev == locus_rev_)
        assert annot_fwd is None or (annot_fwd == annot_fwd_)
        assert locus_fwd is None or (locus_fwd == locus_fwd_)

        if not(annot_rev) is None:
            s = "ok %s/-\t" % (igvstr,)
            if not(annot_rev is None):
                s += annot_rev
            if not(locus_rev is None):
                s += ":" + locus_rev
            print(s)

        if not(annot_fwd) is None:
            s = "ok %s/+\t" % (igvstr,)
            if not(annot_fwd is None):
                s += annot_fwd
            if not(locus_fwd is None):
                s += ":" + locus_fwd
            print(s)
        
    except AssertionError:
        print('Expected: %s:%s / %s:%s' % (annot_rev,locus_rev,annot_fwd,locus_fwd))
        print(df_.transpose())

def check_a(igvstr, locus=None):
    (chrom, start, end) = yp.parse_igvstr(igvstr)
    df_ = df_regl.query("chrom == '%s' & start == %d & end == %d" % (chrom, start, end))
    assert len(df_) == 1

    locus_ = df_['associated_locus_id'].tolist()[0]

    try:
        assert locus == locus_
        print("ok %s/.\t%s" % (igvstr, locus))
        
    except AssertionError:
        print('Expected: %s' % (locus,))
        print(df_.transpose())

#> rfc-4 (black element - promoter inside masked??)
#> zen-4 (black element - should be promoter?)sonRNA, I guess could mask promoter
#Rare but compelling cases of bidirectional promoters where one direction was protein-coding, and the other was an annotated non-coding RNA. Addressed by relaxing the non-coding RNA annotation. They're now treated more similarly to protein-coding annotations, so a site can be annotated as protein_coding on one strand, and snoRNA on the other strand. (Previously, any site that overlapped a non-coding RNA was excluded from any other annotation...)
#> attf-2 (black element maybe promoter?)
#Now annotated as a promoter for F09G2.2 (protein-coding gene, forward strabd), and F09G2.14 (snoRNA, reverse strand). Could also be a promoter for attf-2, but can't think of much one could do about this give current fragmented long cap data.
check_p('chrIII:6971981-6972101', 'non-coding_RNA', 'coding_promoter', 'mir-5549', 'rfc-4')
check_p('chrIV:6117863-6118043', 'non-coding_RNA', 'coding_promoter', 'M03D4.78', 'zen-4')
check_p('chrV:7196589-7196718', 'non-coding_RNA', 'coding_promoter', 'F09G2.14', 'F09G2.2') # Could be attf-2 reverse-strand promoter?

#> csc-1
#> syx-4
#Compelling promoters; captured by relaxing the short cap threshold for calling a promoter that overlaps a first exon to having a single reproducible cap stack.
check_p('chrII:13596635-13596738', 'coding_promoter', 'coding_promoter', 'fbxc-23', 'csc-1')
check_p('chrIV:8457013-8457189', annot_fwd='coding_promoter', locus_fwd='syx-4')

#> fbxc-15
#Fixed by tweaking the long cap jump test.
check_p('chrII:979338-979455', annot_fwd='coding_promoter', locus_fwd='fbxc-15')

#> sri-5 (promoter annotated to rsr-1 only)
#> C25F9.6 (promoter annotate to C25F9.6 only) - looks like there is jump in txn?
#> F08F3.8 (promoter annotated to acl-6 only) - looks like there is longcap signal
#> rsd-2 (maybe one of the yellow elements promoter?) Looks like there is txn from furthest right yellow element
#Weak outron signal; now captured by requiring 1 long cap tag in each replicate, and at least 3 tags total (previously at least 2 tags in both replicates).
check_p('chrI:11182502-11182624', 'coding_promoter', 'coding_promoter', 'rsr-1', 'sri-5')
check_p('chrV:19405507-19405707', 'coding_promoter', 'coding_promoter', 'C25F9.6', 'C25F9.10')
check_p('chrV:5433170-5433353', 'coding_promoter', 'coding_promoter', 'F08F3.8', 'acl-6')
check_p('chrIV:13541820-13541934', annot_rev='coding_promoter', locus_rev='rsd-2')

# Sites downstream within a long exon that should not be called promoters
check_a('chrII:7150014-7150236', 'T07F8.1')
check_a('chrV:16262064-16262178', 'nhr-127')
check_a('chrI:2890703-2890828', 'Y71F9AL.6,arx-1')
check_a('chrI:183422-183568', 'atm-1')

# "Long distance" ncRNA annotations (should not happen)
check_a('chrII:6891890-6892040', 'C44B7.12')

# Located between multiple alternative transcripts on the forward strand; looks like a true promoter (fails currently)
check_p('chrII:6486433-6486590', 'coding_promoter', 'coding_promoter', 'T28D9.9', 'T28D9.3')

# addressed via closest_exon_flank ~< 250 (ncRNA downstream of site; exon1 from upstream gets mis-assigned)
check_p('chrI:111191-111378', annot_fwd='non-coding_RNA', locus_fwd='F53G12.12')

# examples near/around first exons -- pass via allowing incr test
check_a('chrII:6897334-6897462', 'C44B7.11') # strong short cap mis-aligned with TSS
check_a('chrII:11415566-11415675', 'C05D12.1') # well positioned ATAC-seq, weak/non-existant scap
check_p('chrII:6775038-6775173', annot_rev='coding_promoter', locus_rev='agr-1') # looks like a true promoter; scap within UTR; 62bp
check_p('chrII:2296175-2296310', annot_rev='coding_promoter', locus_rev='fbxc-25') # true promoter; scap 126bp from TSS; all within UTR
check_p('chrII:1818315-1818462', annot_rev='coding_promoter', locus_rev='fbxb-10') # true promoter but 3 reads within netural zone
check_p('chrI:5486294-5486413', 'coding_promoter', 'coding_promoter', 'rpl-19', 'C09D4.3') # allow raft approach for downstream?

# pseudogene promoters
check_p('chrII:14563893-14564084', annot_rev='pseudogene_promoter', locus_rev='C01G12.12')

42245 regions loaded
ok chrIII:6971981-6972101/-	non-coding_RNA:mir-5549
ok chrIII:6971981-6972101/+	coding_promoter:rfc-4
ok chrIV:6117863-6118043/-	non-coding_RNA:M03D4.78
ok chrIV:6117863-6118043/+	coding_promoter:zen-4
ok chrV:7196589-7196718/-	non-coding_RNA:F09G2.14
ok chrV:7196589-7196718/+	coding_promoter:F09G2.2
ok chrII:13596635-13596738/-	coding_promoter:fbxc-23
ok chrII:13596635-13596738/+	coding_promoter:csc-1
ok chrIV:8457013-8457189/+	coding_promoter:syx-4
ok chrII:979338-979455/+	coding_promoter:fbxc-15
ok chrI:11182502-11182624/-	coding_promoter:rsr-1
ok chrI:11182502-11182624/+	coding_promoter:sri-5
ok chrV:19405507-19405707/-	coding_promoter:C25F9.6
ok chrV:19405507-19405707/+	coding_promoter:C25F9.10
ok chrV:5433170-5433353/-	coding_promoter:F08F3.8
ok chrV:5433170-5433353/+	coding_promoter:acl-6
ok chrIV:13541820-13541934/-	coding_promoter:rsd-2
ok chrII:7150014-7150236/.	T07F8.1
ok chrV:16262064-16262178/.	nhr-127
ok chrI:2890703-2890828/.	Y71F9AL.6,arx-1
ok chrI:

In [4]:
# Missed promoters close to 5' UTR
# Working proposal:
# - classify annotated TSSs as 'transcribed' if there's short cap [-100;200]
# - 'jump test' within-UTR accessible sites are promoters if the TSS is 'transcribed'
# (Alternatives: require short cap mode to precede the coding region? use "coding part" of the first exon?)

# short cap mode off by 1bp
#check_p('chrII:2772201-2772322', annot_rev='coding_promoter', locus_fwd='sdz-10')

# true promoter; scap does not match UTR annotation
#check_p('chrII:2320235-2320376', annot_fwd='coding_promoter', locus_fwd='ZK1240.3')

# potential forward promoter (although signal *is* noisy), except UTR extends upstream of the site...
#check_p('chrII:1821387-1821512', annot_fwd='coding_promoters', locus_fwd='fbxc-19', annot_rev='coding_promoter', 'locus_rev'='fbxa-3'

# Reverse strand: should be eft-3 instead of miRNA mir-5549?
#check_p('chrIII:6971981-6972101', annot_fwd='coding_promoters', locus_fwd='rfc-4', annot_rev='coding_promoter', locus_rev='eft-3')

Expected: coding_promoter:eft-3 / coding_promoters:rfc-4
                                                   16050
chrom                                             chrIII
start                                            6971980
end                                              6972101
annot                                    coding_promoter
annot_fwd                                coding_promoter
annot_rev                                 non-coding_RNA
promoter_gene_id_fwd                      WBGene00004340
promoter_locus_id_fwd                              rfc-4
promoter_gene_biotype_fwd                 protein_coding
promoter_gene_id_rev                      WBGene00219220
promoter_locus_id_rev                           mir-5549
promoter_gene_biotype_rev                          miRNA
associated_gene_id         WBGene00004340,WBGene00219220
associated_locus_id                       mir-5549,rfc-4
atac_mode                                        6972040
atac_wt_emb_height             

In [ ]:
# Differences to previous github version
fp_head_bed = 'annot/S2_regulatory_annotation/S2_regulatory_annotation.bed_HEAD.bed'
fp_head_tsv = 'annot/S2_regulatory_annotation/S2_regulatory_annotation.tsv_HEAD.tsv'
!git show HEAD:{fp_regl} > {fp_head_tsv}
!git show HEAD:annot/S2_regulatory_annotation/S2_regulatory_annotation.bed > {fp_head_bed}

# Diff against a specific earlier version
#fp_head_bed = 'annot/S2_regulatory_annotation/S2_regulatory_annotation.bed_1030.bed'
#fp_head_tsv = 'annot/S2_regulatory_annotation/S2_regulatory_annotation.tsv_1030.tsv'
#!git show 2040a6:annot/S2_regulatory_annotation/S2_regulatory_annotation.bed > {fp_head_bed}
#!git show 2040a6:annot/S2_regulatory_annotation/S2_regulatory_annotation.tsv > {fp_head_tsv}

df_head = pd.read_csv(fp_head_tsv, sep='\t')
m_diff_ = (df_head['annot_fwd'] != df_regl['annot_fwd']) | (df_head['annot_rev'] != df_regl['annot_rev'])# \
#| ((df_head['promoter_gene_id_fwd'] != df_regl['promoter_gene_id_fwd'])) \
#| ((df_head['promoter_gene_id_rev'] != df_regl['promoter_gene_id_rev'])) \
#| ((df_head['associated_gene_id'] != df_regl['associated_gene_id']))
print('%d regions annotated differently compared to last commit' % (sum(m_diff_)))

In [ ]:
# Randomly sample altered regions
df_diff = df_regl[['chrom', 'start', 'end']].copy()
df_diff['label_head'] = df_head['label']
df_diff['label_regl'] = df_regl['label']
df_diff['annot_rev_head'] = df_head['annot_rev']
df_diff['annot_rev_regl'] = df_regl['annot_rev']
df_diff['annot_fwd_head'] = df_head['annot_fwd']
df_diff['annot_fwd_regl'] = df_regl['annot_fwd']
yp.df_sample(df_diff.loc[m_diff_], 50)

In [ ]:
# Write a .bed-file of all the regions that changed
fp_diff = 'annot/S2_regulatory_annotation/S2_regulatory_annotation_diff.bed'
df_diff[m_diff_][['chrom', 'start', 'end']].to_csv(fp_diff, index=False, header=False, sep='\t')
!wc -l {fp_diff}

Possible additional improvements...

In [ ]:
# exon2 overlap rule is too conservative & masks even when txn clearly does not originate from exon
# => test for jump/incr at exon boundary & discard from annotation only if it tests positive
#check_annot_summary('chrIV:13588263-13588441', 'Y45F10B.12') trans-spliced promoter in L4?

In [ ]:
# extra long outrons???
# snt-2 has a distal promoter spanning 40kb and multiple genes (chrIII:787,433-829,218)
# => cut-off at e.g. 10kb?
#check_annot_summary('chrIII:828052-828187', '??')
#chrIII:4369629-4369711

In [ ]:

#check_annot_summary('chrII:11371902-11372117', '(C33B4.2)') # short cap 100bp downstream of annotated coding region
#check_annot_summary('chrI:5485952-5486062', 'rpl-19') # promoter assigned to un-translated tiny exon; scap not align
#check_annot_summary('chrII:11347905-11348076', 'B0491.6 / B0491.1') # long UTR; short cap precedes TSS
#check_annot_sumary('chrII:1818315-1818462', 'fbxb-10') # true promoter; scap ~60bp chrII:1821387-1821512
#check_annot_summary('chrIII:5876290-5876433', 'aldo-2')

In [ ]:
#> lir-2 (green element may be promoter?) Looks like a promoter, I think
#check_annot_summary('chrII:7667887-7668072', 'lir-2') # Tricky region, as short cap does not align with 5'